In [1]:
#PROGRAM - Cell tracking group Assignment COMP9571 UNSW
#
#PROGRAMMERS 
# - John Cawood (z5117605) segementation D2 and D3
# - Alek Petkovski (z5164896) tracking, mitosis, motion analysis of cell
# - Sankalp Shukla segementation D1
#
#LAST MODIFIED DATE - 20/07/2020
#
#USAGE - enter the folder pathway when prompted that contains the cell images
#        enter an integer that defines the cell type as promted
#        press keyboard key to move to next image in image sequence
#
#DESCRIPTION - Tracks cells and draws a bounding around each cell
#              provides number of cells in each image as an output to terminal 
#              keeps a running trajectory of the movement of cells
#
#---------------------------PACKAGES----------------------------------------------------------------
import numpy as np
import cv2
from matplotlib import pyplot as plt
import imutils
import os
from scipy.spatial import distance
from classes.centroid_tracker_and_mitosis import CentroidTrackerAndMitosis

import imutils
import random as rng
from PIL import Image

from scipy import ndimage as ndi
from skimage import morphology
from skimage.morphology import watershed
from skimage.feature import peak_local_max
from sklearn.cluster import MeanShift
from skimage.filters import threshold_otsu
from skimage.filters import gaussian
from skimage.segmentation import active_contour
#---------------------------------------------------------------------------------------------------

#datasets need different pre processing
#DIC-C2DH-HeLa-----> dataset = 1
#Fluo-N2DL-HeLa----> dataset = 2
#PhC-C2DL-PSC -----> dataset = 3

In [2]:
def segmentation(img,dataset):
    #define elipse based kernel as base
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,  (15,15))

    #different preproccessing techniques for different cells
    if(dataset == 3):
        tophat = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, kernel)
        gray = cv2.cvtColor(tophat, cv2.COLOR_BGR2GRAY)
    else:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
    #gaussian blur and then otsu thresholding to reduce grey noise in the cells and improve watershed segmentation
    #results
    blur = cv2.GaussianBlur(gray,(3,3),0)
    ret,thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    #get rid of artifacts that are too small to be cells
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,  (5,5))
    thresh = cv2.erode(thresh,kernel,iterations=1)
    thresh = cv2.dilate(thresh,kernel,iterations=1)
    thresh = cv2.dilate(thresh,kernel,iterations=1)
    thresh = cv2.erode(thresh,kernel,iterations=1)

    #on the PhC-C2DL-PSC dataset doing another erosion allows better sepperation of close together cells
    if dataset == 3:
        thresh = cv2.erode(thresh,kernel,iterations=1)

    return thresh

In [3]:
def create_box(img,dataset,thresh,filename,pathway_mask):
    parameters = []
    
    centroid = np.zeros(img.shape, np.uint8)
    #perform watershedding segmentation
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,  (3,3))
    background = cv2.dilate(thresh,kernel,iterations=1)
    uncertain_area = cv2.subtract(background,thresh)

    ret, markers = cv2.connectedComponents(thresh)

    markers = markers+1
    markers[uncertain_area==255] = 0
    
    markers = cv2.watershed(img,markers)
    img[markers == -1] = [255,0,0]

    unique_markers = np.unique(markers)
    #print the number of cells in the image to terminal output
    #need to subtract 2 to get number of cells since background is counted twice
    print("Number of Cells in Image("+filename+"): "+ str(len(unique_markers)-2)+"\n")

    #iterate over segmented markers and draw bounding boxes for each cell
    #uses the centroid of the bound to show cell trajectory on a sepperate mask

    for m in unique_markers:
        if m == -1 or m == 1:
            continue
        mask = np.zeros((img.shape[0], img.shape[1]), dtype="uint8")
        mask[markers == m] = 255
        #find contours
        contours = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = imutils.grab_contours(contours)
        c = max(contours, key=cv2.contourArea)

        #create bounding box for cell
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
        contours_poly = cv2.approxPolyDP(c, 3, True)
        area_org = cv2.contourArea(c)

        #to find more accurate area 
        rect1 = cv2.minAreaRect(contours_poly)
        box1 = cv2.boxPoints(rect1)
        box1 = np.int0(box1)
        area = cv2.contourArea(box1)

        X_n = x+w
        Y_n = y+h
        weighted_cent_x = int((x+X_n)/2)
        weighted_cent_y = int((y+Y_n)/2)

        #create bounding box for cell
        (x, y, w, h) = cv2.boundingRect(c)
        parameters.append((weighted_cent_x,weighted_cent_y,area,x,y,X_n,Y_n))

        #show centroid of the cells bounding box on the pathway image
        pathway_color = [255, 0, 0]
        pathway_mask[int(y+h/2),int(x+w/2)] = (pathway_color[0],pathway_color[1],pathway_color[2])
        centroid[int(y+h/2),int(x+w/2)] = (pathway_color[0],pathway_color[1],pathway_color[2])

    #update the pathway color to differentiate between timesteps on the pathway mask
    if pathway_color[1] != 255:
        pathway_color[1] = pathway_color[1]+1
    elif (pathway_color[2] != 255):
        pathway_color[2] = pathway_color[2]+1
    else:
        pathway_color = (255, 0 ,0)
    
    return (parameters, pathway_mask)

In [4]:
def create_box_dataset_1(frame,img,pathway_mask,sequence_number):
    counter = 0
    
    ### Kernel for morphological tranformation ###
    kernel = np.ones((5,5),np.uint8)
    
    ### Blurring to smooth the insides of the cells ###
    ### Choose the appropriate blur ###
    ## LOOK AT FILTER 2D
    cl = cv2.equalizeHist(frame)
    cl = cv2.medianBlur(cl, 17)
    cl = cv2.fastNlMeansDenoising(cl, None, 9, 19, 19)
    
    ### Applying the proper threshold to the h-dome image to obtain a binary image ###
    ### Choose the appropriate threshold function ###
    thresh = cv2.adaptiveThreshold(cl,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,191,3)
    thresh = cv2.bitwise_not(thresh)
    
    ### Binary image manipulation to remove artifacts ###
    imglab = morphology.label(thresh)
    cleaned = morphology.remove_small_objects(imglab, min_size=800, connectivity=1)
    img3 = np.zeros((cleaned.shape))
    img3[cleaned > 0] = 255 
    img3= np.uint8(img3)
    ### Assigning the final image to 'thresh' variable ###
    thresh = cv2.bitwise_not(img3)
    
    ### Removing artifacts from the threshold image ###
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    thresh = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    
    ### Opening the image obtained from the neural net to use as a mask ###
    ### The neural net identifies the outside borders of the cell sequence and image manipulation identifies ###
    ### the inner borders. Using the neural net image as a masking image allows for seperation of cells ###
    im = Image.open("cell_Images/Dataset 1/Sequence "+str(sequence_number)+" Results/%d_predict.png"%counter)
    print("%d_predict.png"%counter)
    counter += 1
    im = np.array(im)
    ### Eroding the border for better final image ###
    im = cv2.erode(im,kernel,iterations = 1)
    ### Opening and closing to remove artifacts ###
    im = cv2.morphologyEx(im, cv2.MORPH_OPEN, kernel)
    im = cv2.morphologyEx(im, cv2.MORPH_CLOSE, kernel)
    
    ### Resizing the threshold image to the same size as the image obtained from the neural net ###
    thresh = cv2.resize(thresh, im.shape[1::-1])
    ### Using the neural net image as a mask ###
    dst = cv2.bitwise_and(im, thresh)
    ### Removing the small objects from the image ###
    imglab = morphology.label(dst)
    cleaned = morphology.remove_small_objects(imglab, min_size=200, connectivity=1)
    img3 = np.zeros((cleaned.shape))
    img3[cleaned > 0] = 255 
    dst = np.uint8(img3)
    
    ### Resizing the initial unmodified image so that the contours can be displayed on a plot ###
    resized = cv2.resize(frame, im.shape[1::-1])
    im2, contours, hierarchy = cv2.findContours(dst, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    ### Removing contours inside contours because a cell cannot be inside another cell ###
    cells = []
    for i, c in enumerate(contours):
        if hierarchy[0][i][3] == -1:
            cells.append(c)
    
    print("Number of Cells in Image("+str(counter)+"): "+ str(len(contours))+"\n")
    ### Obtaining the coordinates for the bounding box ###
    contours_poly = [None]*len(cells)
    boundRect = [None]*len(cells)
    for i, c in enumerate(cells):
        contours_poly[i] = cv2.approxPolyDP(c, 3, True)
        boundRect[i] = cv2.boundingRect(contours_poly[i])
        
    parameters = []
    ### Drawing the contours and the boundin box ###
    ### Comment out the 'cv2.drawContours' to not draw the cell contours ###
    for i in range(len(cells)):
        color = (0, 255, 0)
        final = cv2.drawContours(resized, contours_poly, i, color)
        final = cv2.rectangle(resized, (int(boundRect[i][0]), int(boundRect[i][1])), \
          (int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])), color, 2)
        
        x = boundRect[i][0]
        y = boundRect[i][1]
        X_n = boundRect[i][0]+boundRect[i][2]
        Y_n = boundRect[i][1]+boundRect[i][3]
        weighted_cent_x = int((x+X_n)/2)
        weighted_cent_y = int((y+Y_n)/2)
        area = (Y_n-y)*(X_n-x)
        parameters.append((weighted_cent_x,weighted_cent_y,area,x,y,X_n,Y_n))
        pathway_color = [255, 0, 0]
        pathway_mask[int(Y_n/2),int(X_n/2)] = (pathway_color[0],pathway_color[1],pathway_color[2])
    
    #update the pathway color to differentiate between timesteps on the pathway mask
    if pathway_color[1] != 255:
        pathway_color[1] = pathway_color[1]+1
    elif (pathway_color[2] != 255):
        pathway_color[2] = pathway_color[2]+1
    else:
        pathway_color = (255, 0 ,0)
   
    return (parameters, pathway_mask, final)

In [5]:
def main(dataset,sequence_number):
    #set a default filepath
    if dataset == 1 : 
        file_path = 'cell_Images/DIC-C2DH-HeLa/Sequence '+str(sequence_number)
    elif dataset == 2:
        file_path = 'cell_Images/Fluo-N2DL-HeLa/Sequence '+str(sequence_number)
    elif dataset == 3:
        file_path = 'cell_Images/PhC-C2DL-PSC/Sequence '+str(sequence_number)

    directory = os.fsencode(file_path)

    # open first image in file to get size for the cell tracking pathway map
    # assumes first file in folder is called "t000.tif"
    (h,w,d) = cv2.imread(file_path+"/t000.tif").shape
    pathway_mask = np.zeros((h,w,3), np.uint8)
    pathway_color = [255, 0, 0]

    # initialise our centroid tracker and frame dimensions
    ct = CentroidTrackerAndMitosis()
    
    N = 1000 # Trace max 1000 cells
    AllColor = []

    for i in range(N):
        color1 = (list(np.random.choice(range(256), size=3)))  
        color =[int(color1[0]), int(color1[1]), int(color1[2])] 
        AllColor.append(color)

    Follow_cell = -1
    AllCoordiantes = [[] for i in range(N)]

    EuclideanDistance = [[] for i in range(N)]
    TotalEuclideanDistance = [0.0 for i in range(N)]
    NetDistance = [0.0 for i in range(N)]
    RatioOfTheCellMotion = [0.0 for i in range(N)]

    # loop over the frames from the video stream
    for file in sorted(os.listdir(directory)):
        filename = os.fsdecode(file)
        frame = cv2.imread(str(file_path+"/"+filename))

        thresh = segmentation(frame,dataset)

        # bounding box rectangles
        rects = []

        if dataset == 2 or dataset == 3:
            para, pw = create_box(frame, dataset, thresh, filename, pathway_mask)
        elif dataset == 1:
            frame1= cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            para, pw, frame = create_box_dataset_1(frame1, frame, pathway_mask,sequence_number)
            
        (centroid, coordiantes, area, mitosis) = ct.update(para, dataset)
        # loop over the tracked objects

        for (cellID, co) in coordiantes.items():
            
            color1 = (list(np.random.choice(range(256), size=3)))  
            color =[int(color1[0]), int(color1[1]), int(color1[2])] 

            AllCoordiantes[cellID].extend([int((co[0]+co[2])/2), int((co[1]+co[3])/2)])
            # draw both the ID of the object and the centroid of the
            # object on the output frame
            text = "{}".format(cellID)
            cv2.putText(frame, text, (co[0] - 2, co[1] - 2), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)
        
        x_Follow_cell = -1
        y_Follow_cell = -1

        c=0
        for cellTraceLine in AllCoordiantes:
            totalED = 0.0
            if len(cellTraceLine) > 2:
                EuclideanDistance[c]=[]
                x0 = cellTraceLine[0]
                y0 = cellTraceLine[1]
                if Follow_cell != -1 and Follow_cell == cellID:
                    x_Follow_cell = cellTraceLine[0]
                    y_Follow_cell = cellTraceLine[1]

                for x1,y1,x2,y2 in zip(cellTraceLine[0::2], cellTraceLine[1::2], 
                                        cellTraceLine[2::2], cellTraceLine[3::2]):
                    line_thickness = 2
                    cv2.line(frame, (x1, y1), (x2, y2),AllColor[c], thickness=line_thickness)
                    cellCentroid1 = np.array([[x1,y1]])
                    cellCentroid2 = np.array([[x2,y2]])
                    eDistance = distance.cdist(cellCentroid1, cellCentroid2, metric='euclidean')
                    totalED += eDistance
                    TotalEuclideanDistance[c] = totalED
                    EuclideanDistance[c].extend([eDistance]) 

                cellCentroid1 = np.array([[x0,y0]])
                cellCentroid2 = np.array([[x2,y2]])    
                nd = distance.cdist(cellCentroid1, cellCentroid2, metric='euclidean')
                NetDistance[c] = nd
                if nd != 0.0: RatioOfTheCellMotion[c] = TotalEuclideanDistance[c]/NetDistance[c]
            if Follow_cell != -1 and Follow_cell == c and EuclideanDistance[c]: 
                print('=======================================')
                print('Speed of the cell ',c,'is ',EuclideanDistance[c][-1][0][0],' pixels/frame')
            if Follow_cell != -1 and Follow_cell == c and TotalEuclideanDistance[c]: 
                print('Total Distance of the cell :',c,'is ',TotalEuclideanDistance[c][0][0])
            if Follow_cell != -1 and Follow_cell == c and NetDistance[c]: 
                print('Net Distance of the cell :',c,'is ',NetDistance[c][0][0])
            if Follow_cell != -1 and Follow_cell == c and RatioOfTheCellMotion[c]: 
                print('Ratio of the cell motion:',c,'is ',RatioOfTheCellMotion[c][0][0])
                print('=======================================')
            c+=1
        for (ID,x,y) in mitosis:
            co = coordiantes[ID]
            cv2.rectangle(frame, (co[0], co[1]), (co[2], co[3]), (255, 255, 255), 2)

        print("Count of Dividing Cells "+str(len(mitosis))) if len(mitosis)!= 0 else print("No Mitosis Detected ")
        #show the results on current image
        #press any key to move to next image
        #shows an ongoing cell trajectory
        print("######################################################")
        
        cv2.namedWindow('image',cv2.WINDOW_AUTOSIZE)
        cv2.moveWindow('image', 20,20)
        cv2.imshow("image", frame)
        #cv2.imshow("pathways", pw)
        key = cv2.waitKey(0)
        if key == ord('p'):
            cnum = input("Enter a cell number: ")
            Follow_cell = int(cnum)
        cv2.destroyAllWindows()

In [ ]:
if __name__ == "__main__":
    #change dataset and sequence number here 
    main(2,1)

Number of Cells in Image(t000.tif): 27

No Mitosis Detected 
######################################################
Number of Cells in Image(t001.tif): 27

Predicted Mitosis 3:[403 173]

Count of Dividing Cells 1
######################################################
Number of Cells in Image(t002.tif): 28

Predicted Mitosis 20:[120 529]
Predicted Mitosis 1:[678  15]

Mother Cell's ID is 3:(403, 173)
Daughter Cell 1 ID is 27:[426 151]
Daughter Cell 2 ID is 28:[402 183]

Count of Dividing Cells 2
######################################################
Number of Cells in Image(t003.tif): 30

Predicted Mitosis 1:[689  30]

Mother Cell's ID is 20:(120, 529)
Daughter Cell 1 ID is 29:[124 550]
Daughter Cell 2 ID is 31:[132 514]

Mother Cell's ID is 1:(678, 15)
Daughter Cell 1 ID is 0:[637  21]
Daughter Cell 2 ID is 32:[689  30]

Count of Dividing Cells 1
######################################################
Number of Cells in Image(t004.tif): 34

Predicted Mitosis 32:[694  33]
Predicted Mitos

Enter a cell number:  12


Number of Cells in Image(t005.tif): 35

Predicted Mitosis 30:[352 166]
Predicted Mitosis 23:[162 566]

Mother Cell's ID is 30:(357, 162)
Daughter Cell 1 ID is 34:[350 129]
Daughter Cell 2 ID is 39:[352 166]

Mother Cell's ID is 11:(1042, 412)
Daughter Cell 1 ID is 36:[1020  391]
Daughter Cell 2 ID is 40:[1042  414]

Mother Cell's ID is 24:(199, 611)
Daughter Cell 1 ID is 35:[208 584]
Daughter Cell 2 ID is 41:[198 612]

Mother Cell's ID is 37:(694, 33)
Daughter Cell 1 ID is 33:[698  15]
Daughter Cell 2 ID is 42:[688  35]

Speed of the cell  12 is  4.47213595499958  pixels/frame
Total Distance of the cell : 12 is  15.23996489063195
Net Distance of the cell : 12 is  12.041594578792296
Ratio of the cell motion: 12 is  1.26561019729668
Count of Dividing Cells 2
######################################################
Number of Cells in Image(t006.tif): 33

Predicted Mitosis 26:[395 669]
Predicted Mitosis 23:[160 579]

Mother Cell's ID is 23:(162, 566)
Daughter Cell 1 ID is 29:[131 553]
Daught